In [1]:
import boto3

In [2]:
dynamodb = boto3.resource('dynamodb', aws_access_key_id='XXXXXXXXXXXXXXXXXXXX', 
            aws_secret_access_key='YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY',)

In [4]:
print('"{dynamodb}" is now being created as a DynamoDB resource object.\n')

"{dynamodb}" is now being created as a DynamoDB resource object.



In [5]:
table = dynamodb.create_table(
    TableName = 'MyMovieList',
    AttributeDefinitions=[
        {'AttributeName': 'rank_id','AttributeType': 'S'},{'AttributeName': 'genre',  'AttributeType': 'S'},    
        {'AttributeName': 'title',  'AttributeType': 'S'},{'AttributeName': 'year',   'AttributeType': 'S'},],
    KeySchema = [{'AttributeName': 'rank_id', 'KeyType': 'HASH'},{'AttributeName': 'genre'  ,'KeyType': 'RANGE'},],
    GlobalSecondaryIndexes=[{'IndexName': 'new_search',
        'KeySchema': [{'AttributeName': 'year', 'KeyType': 'HASH'},{'AttributeName': 'title','KeyType': 'RANGE'},],
        'Projection': {'ProjectionType': 'INCLUDE', 'NonKeyAttributes': ['rank_id','genre']},
        'ProvisionedThroughput':{'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5},}],    
    ProvisionedThroughput={'ReadCapacityUnits': 5,'WriteCapacityUnits': 5},
    Tags=[{'Key': 'Project 9', 'Value': 'Group B - DynamoDB project'},],
)
table.wait_until_exists() # Wait until the table exists.
print(table.item_count)   # Print the counting number of items
print(f"The date and time of creating this table: {table.creation_date_time}")
print("===============\n")

0
The date and time of creating this table: 2023-04-18 01:03:32.648000-04:00



In [6]:
import csv             # import a 'cvs' Python library to access to CSV files.
entire_movie_list=[]   # Initialize as an empty list.
eachRow=[]             # eachRow list will consist of 3 items; Rank_ID, Title, and Year.
with open('movie_table.csv', newline='') as csvfile:    # open movie_table.csv file and read by each entire new line, 
                                                        # then designate it as an csvfile object.
    fileReader = csv.reader(csvfile, delimiter= ",", quotechar = None)  # See comma delimiter as a separator 
                                                                        # to recognize individual items
    for eachRow in fileReader:  # eachRow is assigned to each new line  
        Item={}                 # Reinitizlie as empty dictionary per each new line
        # Add 3 individual items per each movie below:
        Item['rank_id'] = eachRow[0]
        Item['genre'] = eachRow[1]
        Item['title'] = eachRow[2]
        Item['year'] = eachRow[3]
        entire_movie_list.append(Item) # Add each movie dictionary to the entire movie list
    print(f"The following movie list will be added to MyMovieList table:\n{entire_movie_list}")


The following movie list will be added to MyMovieList table:
[{'rank_id': '2', 'genre': 'Drama', 'title': 'The God Father', 'year': '1972'}, {'rank_id': '6', 'genre': 'Romance', 'title': 'Gone with the Wind', 'year': '1939'}, {'rank_id': '8', 'genre': 'Drama', 'title': " Schindler's List", 'year': '1993'}, {'rank_id': '10', 'genre': 'Musical', 'title': ' The Wizard of Oz', 'year': '1939'}, {'rank_id': '15', 'genre': 'Adventure', 'title': 'Star Wars', 'year': '1977'}, {'rank_id': '40', 'genre': 'Musical', 'title': 'The Sound of Music', 'year': '1965'}, {'rank_id': '41', 'genre': 'Adventure', 'title': 'King Kong', 'year': '1933'}, {'rank_id': '57', 'genre': 'Drama', 'title': 'Rocky', 'year': '1976'}, {'rank_id': '74', 'genre': 'Drama', 'title': 'The Silence of the Lambs', 'year': '1991'}, {'rank_id': '100', 'genre': 'Epic', 'title': 'Ben-Hur', 'year': '1959'}]


In [7]:
import boto3
dynamodb = boto3.resource('dynamodb') # Get the service resource.
table = dynamodb.Table('MyMovieList') # call your MyMovieList DynamoDB table
# add a list of 10 movie dictionaries to MyMovieList DynamoDB table.
for i in range(len(entire_movie_list)):
    table.put_item(Item=entire_movie_list[i])
print("Completed to add all your movie dictionaries to your DynamoDB table...")
print(f"The number of your table items: {table.item_count}")
print(f"The entire table below:\n{table}")


Completed to add all your movie dictionaries to your DynamoDB table...
The number of your table items: 0
The entire table below:
dynamodb.Table(name='MyMovieList')


In [8]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
dynamodb = boto3.client('dynamodb') # Get the service resource.
#table = dynamodb.Table('MyMovieList') # call your MyMovieList DynamoDB table
response = dynamodb.scan(
    ExpressionAttributeNames={
        '#RI':'rank_id', '#G': 'genre', '#T': 'title', '#Y': 'year'},
    ExpressionAttributeValues={
        ':alldrama': {'S': 'Drama',}, 
    },
    FilterExpression='genre = :alldrama',
    ProjectionExpression='#T, #Y, #RI, #G',     
    TableName = 'MyMovieList'
)
items = response['Items']
print(items)

[{'rank_id': {'N': '8'}, 'year': {'N': '1993'}, 'genre': {'S': 'Drama'}, 'title': {'S': " Schindler's List"}}, {'rank_id': {'N': '2'}, 'year': {'N': '1972'}, 'genre': {'S': 'Drama'}, 'title': {'S': 'The God Father'}}, {'rank_id': {'N': '57'}, 'year': {'N': '1976'}, 'genre': {'S': 'Drama'}, 'title': {'S': 'Rocky'}}, {'rank_id': {'N': '74'}, 'year': {'N': '1991'}, 'genre': {'S': 'Drama'}, 'title': {'S': 'The Silence of the Lambs'}}]
